In [15]:
import numpy as np
from scipy import *
import sys, os
import pickle

In [75]:
from matplotlib.patches import Ellipse
def plotEllipse(pos,P,edge,ls,ilabel,ax):
    '''plot the contour for covariance p
    where position is the center position
    p is the covariance [2,2] matirx'''
    U, s, Vh = svd(P) 
    orient = math.atan2(U[1,0],U[0,0])*180/pi
    ellipsePlot = Ellipse(xy=pos, width=2.0*math.sqrt(s[0]), 
                          height=2.0*math.sqrt(s[1]), angle=orient,edgecolor=edge, fill = 0, label=ilabel,ls=ls,linewidth=1.5)
    ax.add_patch(ellipsePlot)
    return ellipsePlot

def Fisher(covI, der):
    '''
    Input: 
    covariance inverse matrix (Nbin, Nbin)
    derivatives (Nparams, Nbin), where Nparams is the number of parameters 
    skyscaling = area_sims / area_actual_survey = 12.25/2e4 for our sim and LSST
    Return the Fisher matrix
    '''
    Nparams, Nbin = der.shape
    F = zeros( shape= (Nparams, Nparams))
    for i in range(Nparams):
        for j in range(Nparams):
            dA, dB = der[i], der[j]
            Mij = mat(dA).T*mat(dB) + mat(dB).T*mat(dA)  
            F [i,j]= 0.5*trace(covI*Mij)
    Ferr = real(sqrt(mat(F).I)) [range(Nparams), range(Nparams)]
    return F, Ferr

In [56]:
labels=['Pm','Bm', 
        'Ph Mh=[12, 12.5]', 'Bh Mh=[12, 12.5]', 
        'Ph Mh=[12.5, 13]', 'Bh Mh=[12.5, 13]', 
        'Ph Mh=[13, 16]', 'Bh Mh=[13, 16]']

In [78]:
params = loadtxt('cosmological_parameters_b1.txt')
mnv, om, As, si8, b1a, b1b, b1c = params.T

nsnaps_arr = loadtxt('nsnap_jia.txt')
nsnaps_arr = delete(nsnaps_arr,39)

Phhz0 = array([load('Ph2h/Ph2h_mnv%.5f_om%.5f_As%.4f_%03d.npy'
                    %(mnv[i],om[i],As[i],nsnaps_arr[i]-1)) for i in range(101)])
idxok=where(~isnan(covs[0]))[0]
Phhz0 = Phhz0[:,:,idxok]
Nok = len(idxok)

print nsnaps_arr.shape, params.shape, Phhz0.shape

(101,) (101, 7) (101, 9, 42)


In [79]:
fn_cov = 'cov/Ph2h_mnv0.00000_om0.30000_As2.1000_065.npy.cov.pickle'
cov_arr = pickle.load(open(fn_cov, "rb"))['diagcov']

covs = [cov_arr[Px][idxok] for Px in labels]
covIs = []
for icov in covs:
    icov2d = zeros((Nok,Nok))
    icov2d[diag_indices(Nok)] = icov
    covIs.append(mat(icov2d).I)
    
# covPm, covBm = cov_arr['Pm'], cov_arr['Bm']
# covPh1, covBh1 = cov_arr['Ph Mh=[12, 12.5]'], cov_arr['Bh Mh=[12, 12.5]']
# covPh2, covBh2 = cov_arr['Ph Mh=[12.5, 13]'], cov_arr['Bh Mh=[12.5, 13]']
# covPh3, covBh3 = cov_arr['Ph Mh=[13, 16]'], cov_arr['Bh Mh=[13, 16]']